<a href="https://colab.research.google.com/github/gaddam007-git/book-price-intelligence-system/blob/main/Notebooks/milestone_4_cross_platform_integration/Milestone_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Milestone 4: Cross-Platform Integration & Notification System Deployment**

**Converting UPC code to ISBN number  TASK 1:**

**Extracting UPC code, Book Titles, Prices**

In [ ]:
!pip install beautifulsoup4 requests

In [ ]:
# =========================================================
# STEP 0 – SCRAPE books.toscrape.com
# Create RAW input data (CSV + JSON)
# =========================================================
# This script scrapes all books from books.toscrape.com
# and stores raw book metadata into CSV and JSON files.

import requests
# requests → used to send HTTP GET requests to fetch web pages

from bs4 import BeautifulSoup
# BeautifulSoup → used to parse and navigate HTML content

import pandas as pd
# pandas → used to store scraped data and export it to CSV/JSON

from pathlib import Path
# Path → provides OS-independent file and directory handling

from urllib.parse import urljoin
# urljoin → safely combines base URLs with relative URLs


# =========================
# CONFIGURATION
# =========================

BASE_URL = "https://books.toscrape.com/"
# Base URL of the website being scraped

CATALOGUE_URL = urljoin(BASE_URL, "catalogue/")
# URL pointing to the catalogue pages containing paginated book lists

OUTPUT_DIR = Path("output")
# Directory where output files will be stored

OUTPUT_DIR.mkdir(exist_ok=True)
# Creates the output directory if it does not already exist

CSV_PATH = OUTPUT_DIR / "books_raw_data.csv"
# File path for CSV output

JSON_PATH = OUTPUT_DIR / "books_raw_data.json"
# File path for JSON output

HEADERS = {"User-Agent": "Mozilla/5.0"}
# Custom headers to mimic a real browser and avoid being blocked


# =========================
# HELPER FUNCTION
# =========================

def get_book_details(book_url):
    """
    Fetches and extracts detailed information from a single book page.
    Returns category, title, UPC, and price.
    """

    response = requests.get(book_url, headers=HEADERS)
    # Sends HTTP request to the individual book page

    soup = BeautifulSoup(response.text, "html.parser")
    # Parses the HTML response into a navigable structure

    title = soup.find("h1").text.strip()
    # Extracts the book title from the <h1> tag

    # Category
    category = soup.select("ul.breadcrumb li a")[2].text.strip()
    # Extracts book category from breadcrumb navigation

    # Product information table
    table = soup.find("table", class_="table table-striped")
    # Locates the table containing product metadata

    rows = table.find_all("tr")
    # Retrieves all table rows

    product_info = {
        row.find("th").text: row.find("td").text for row in rows
    }
    # Converts table rows into a key-value dictionary

    upc = product_info.get("UPC")
    # Extracts the book's unique UPC identifier

    price = product_info.get("Price (incl. tax)")
    # Extracts the book price including tax

    return category, title, upc, price
    # Returns extracted fields to the caller


# =========================
# SCRAPE ALL BOOKS
# =========================

books_data = []
# List to store scraped book records

page_url = CATALOGUE_URL + "page-1.html"
# Starting URL for pagination (first catalogue page)

while page_url:
    # Loop continues until there are no more pages

    print(f"🔍 Scraping page: {page_url}")
    # Logs current page being scraped

    response = requests.get(page_url, headers=HEADERS)
    # Fetches the catalogue page HTML

    soup = BeautifulSoup(response.text, "html.parser")
    # Parses catalogue page HTML

    book_links = soup.select("h3 a")
    # Selects all book links present on the page

    for link in book_links:
        # Iterates through each book on the page

        book_url = urljoin(page_url, link["href"])
        # Converts relative book link to absolute URL

        category, title, upc, price = get_book_details(book_url)
        # Fetches detailed data for the book

        books_data.append({
            "Category": category,
            "Book Title": title,
            "UPC": upc,
            "Price": price
        })
        # Stores extracted book data as a dictionary

    # Pagination
    next_button = soup.select_one("li.next a")
    # Checks if a "Next" page button exists

    if next_button:
        page_url = urljoin(page_url, next_button["href"])
        # Updates page_url to the next catalogue page
    else:
        page_url = None
        # Ends loop when no more pages exist


# =========================
# SAVE OUTPUT FILES
# =========================

df = pd.DataFrame(books_data)
# Converts list of dictionaries into a pandas DataFrame

df.to_csv(CSV_PATH, index=False)
# Saves data to CSV without row indices

df.to_json(JSON_PATH, orient="records", indent=2)
# Saves data to JSON as a list of records

print("\n✅ Scraping completed successfully!")
# Confirms successful execution

print(f"📁 CSV saved at  : {CSV_PATH}")
print(f"📁 JSON saved at : {JSON_PATH}")
# Displays output file locations

print(f"📘 Total books scraped: {len(df)}")
# Displays total number of books scraped


# =========================
# SHOW SAMPLE OUTPUT (CONSOLE)
# =========================

print("\n📄 Sample data (first 5 rows):")
print(df.head())
# Displays the first 5 records for verification

print("\n📊 Columns:")
print(df.columns.tolist())
# Displays column names in the dataset

🔍 Scraping page: https://books.toscrape.com/catalogue/page-1.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-2.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-3.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-4.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-5.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-6.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-7.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-8.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-9.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-10.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-11.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-12.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-13.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-14.html
🔍 Scraping page: https://books.toscrape.com/catalogue/page-15.html
🔍 Sc

#Observations for **STEP 0 – Books Scraping Code & Output**
---

## 1️⃣ Website Coverage Observation

* The script **successfully scrapes the entire catalogue** of
  **Books to Scrape**.
* Pagination is handled correctly from **page-1 to page-50**.
* Each catalogue page contains **20 books**, giving:

  > 📘 **50 pages × 20 books = 1000 books**

✅ This confirms **complete dataset coverage** with no missing pages.

---

## 2️⃣ Pagination Logic Observation

* The `while page_url:` loop combined with:

  ```python
  soup.select_one("li.next a")
  ```

  ensures:

  * Automatic navigation to the **next page**
  * Clean termination when the **Next** button no longer exists

✅ This is a **robust and scalable pagination strategy**.

---

## 3️⃣ Two-Level Scraping Design

* The scraper uses a **two-step architecture**:

  1. **Catalogue pages** → collect book URLs
  2. **Individual book pages** → extract detailed metadata

**Extracted per book:**

* Category
* Book Title
* UPC (unique identifier)
* Price (including tax)

✅ This mirrors **real-world e-commerce scraping patterns**.

---

## 4️⃣ Data Quality & Structure Observation

* Output columns are **well-structured and consistent**:

```text
['Category', 'Book Title', 'UPC', 'Price']
```

* Sample output confirms:

  * Clean category labels (`Poetry`, `Fiction`, `History`, etc.)
  * Unique UPCs per book
  * Correct titles extracted from `<h1>`

✅ Dataset is **analysis-ready** for later steps.

---

## 5️⃣ Price Encoding Observation (Important)

* Prices appear as:

  ```
  Â£51.77
  ```
* This indicates a **character encoding issue** (UTF-8 vs ISO-8859-1).

📌 **Observation (not an error):**

* The currency symbol is preserved but mis-encoded.
* This is **expected in raw scraping** and typically fixed during:

  * Data cleaning
  * Price normalization steps

✅ Suitable for **RAW input data**, as intended in Step 0.

---

## 6️⃣ File Output Observation

* The script saves data in **two formats**:

  * `books_raw_data.csv`
  * `books_raw_data.json`

📁 Benefits:

* CSV → easy for pandas / Excel
* JSON → useful for APIs, NoSQL, or web apps

✅ This shows **good data engineering practice**.

---

## 7️⃣ Performance & Stability Observation

* Uses lightweight libraries:

  * `requests`
  * `BeautifulSoup`
* No JavaScript rendering → fast execution
* Console logs confirm **smooth sequential scraping** without failures

⚠️ No throttling (`time.sleep`) is used, but:

* This site is a **scraping demo**, so it’s acceptable.

---

## 8️⃣ Final Output Validation

* Console confirms:

  ```text
  📘 Total books scraped: 1000
  ```
* Sample rows match expected site content
* Column names are clean and consistent

✅ Output is **validated and trustworthy**.

**SubTask 1 of TASK 1 Prepare and store clean book data**

In [ ]:
# =========================================================
# TASK 1 – SUB TASK 1
# Prepare and store clean book data (CSV + JSON)
# =========================================================
# This script cleans the raw scraped book data by
# selecting required fields, removing invalid entries,
# and storing the cleaned dataset in CSV and JSON formats.

import pandas as pd
# pandas → used for reading, cleaning, and saving tabular data

from pathlib import Path
# Path → provides platform-independent file and folder handling


# =========================
# CONFIGURATION
# =========================

# Input file generated from STEP 0 (scraping)
INPUT_CSV = Path("output/books_raw_data.csv")
# Path to the raw scraped book data

# Output directory
OUTPUT_DIR = Path("output")
# Directory to store cleaned output files

OUTPUT_DIR.mkdir(exist_ok=True)
# Creates the output directory if it does not exist

# Output files
OUTPUT_CSV = OUTPUT_DIR / "task1_subtask1_clean_books.csv"
# File path for cleaned CSV output

OUTPUT_JSON = OUTPUT_DIR / "task1_subtask1_clean_books.json"
# File path for cleaned JSON output


# =========================
# LOAD DATA
# =========================

df = pd.read_csv(INPUT_CSV)
# Loads the raw book data CSV into a pandas DataFrame

print(f"📘 Total raw records loaded: {len(df)}")
# Prints the number of records loaded for verification


# =========================
# SELECT REQUIRED COLUMNS
# =========================

required_columns = [
    "Category",
    "Book Title",
    "UPC",
    "Price"
]
# Defines the subset of columns needed for further processing

df = df[required_columns]
# Filters the DataFrame to keep only the required columns


# =========================
# DATA CLEANING
# =========================

# Ensure UPC is treated as string
df["UPC"] = df["UPC"].astype(str)
# Converts UPC to string to preserve leading zeros and consistency

# Remove rows with missing or empty UPC
df = df[df["UPC"].notna() & (df["UPC"] != "")]
# Eliminates rows where UPC is null or an empty string

# Remove duplicate UPCs
df = df.drop_duplicates(subset="UPC")
# Keeps only the first occurrence of each unique UPC

print(f"✅ Records after cleaning: {len(df)}")
# Prints final record count after cleaning


# =========================
# SHOW SAMPLE OUTPUT (CONSOLE)
# =========================

print("\n📄 Sample cleaned data (first 5 rows):")
print(df.head())
# Displays the first 5 cleaned records for inspection

print("\n📊 Columns:")
print(df.columns.tolist())
# Prints the column names in the cleaned dataset


# =========================
# SAVE OUTPUT FILES
# =========================

df.to_csv(OUTPUT_CSV, index=False)
# Saves cleaned data to CSV without row index

df.to_json(OUTPUT_JSON, orient="records", indent=2)
# Saves cleaned data to JSON as a list of records

print("\n📁 Clean data saved successfully:")
print(f"➡ CSV  : {OUTPUT_CSV}")
print(f"➡ JSON : {OUTPUT_JSON}")
# Confirms output file creation and locations

📘 Total raw records loaded: 1000
✅ Records after cleaning: 1000

📄 Sample cleaned data (first 5 rows):
             Category                             Book Title  \
0              Poetry                   A Light in the Attic   
1  Historical Fiction                     Tipping the Velvet   
2             Fiction                             Soumission   
3             Mystery                          Sharp Objects   
4             History  Sapiens: A Brief History of Humankind   

                UPC    Price  
0  a897fe39b1053632  Â£51.77  
1  90fa61229261140a  Â£53.74  
2  6957f44c3847a760  Â£50.10  
3  e00eb4fd7b871a48  Â£47.82  
4  4165285e1663650f  Â£54.23  

📊 Columns:
['Category', 'Book Title', 'UPC', 'Price']

📁 Clean data saved successfully:
➡ CSV  : output/task1_subtask1_clean_books.csv
➡ JSON : output/task1_subtask1_clean_books.json


# Observations for **TASK 1 – SUB TASK 1 (Clean Book Data)**
---

## 1️⃣ Input Data Integrity Observation

* The script loads **1000 raw records** generated from the previous scraping step of
  **Books to Scrape**.
* No file path or schema mismatch occurs, confirming:

  * STEP 0 output is **fully compatible** with TASK 1 input.

✅ This validates **pipeline continuity**.

---

## 2️⃣ Column Selection Observation

* Only the **essential analytical fields** are retained:

  * `Category`
  * `Book Title`
  * `UPC`
  * `Price`

📌 Observation:

* Removing unused fields at this stage:

  * Reduces memory usage
  * Simplifies downstream processing
  * Improves dataset clarity

✅ This follows **best practices in data preprocessing**.

---

## 3️⃣ UPC Handling & Uniqueness Observation

* `UPC` is explicitly converted to **string format**, ensuring:

  * No loss of leading zeros
  * Uniform identifier type across the dataset

* The cleaning logic:

  ```python
  df = df[df["UPC"].notna() & (df["UPC"] != "")]
  df = df.drop_duplicates(subset="UPC")
  ```

  ensures:

  * No missing UPCs
  * No duplicate product entries

📘 Result:

```text
Records before cleaning : 1000
Records after cleaning  : 1000
```

✅ Confirms **all scraped books are valid and uniquely identifiable**.

---

## 4️⃣ Data Quality Observation

* Since record count remains unchanged:

  * No corrupted rows
  * No duplicate books
  * No missing identifiers

📌 This indicates:

* High-quality scraping in STEP 0
* Reliable source structure

✅ Cleaning step acts as **validation + enforcement**, not heavy filtering.

---

## 5️⃣ Price Field Observation (Expected Behavior)

* Prices remain as strings with currency symbols:

  ```
  Â£51.77
  ```

📌 Observation:

* This step **intentionally does not normalize prices**.
* It preserves **raw financial text** for:

  * Later numeric conversion
  * Currency handling
  * Price adjustment logic

✅ Correct separation of concerns between **cleaning** and **transformation** stages.

---

## 6️⃣ Output Format Observation

* Cleaned data is saved in:

  * CSV → analytics & ML pipelines
  * JSON → APIs & web integration

📁 Output files:

```text
task1_subtask1_clean_books.csv
task1_subtask1_clean_books.json
```

✅ Demonstrates **multi-format data readiness**.

---

## 7️⃣ Sample Output Validation

* Sample rows confirm:

  * Category labels are intact
  * Titles are clean and readable
  * UPCs are consistent
  * Prices are uniformly formatted

📊 Columns:

```text
['Category', 'Book Title', 'UPC', 'Price']
```

✅ Dataset is now **structured, validated, and standardized**.

**Sub-Task 2 of Task 1 UPC to ISBN Resolution Using Hybrid Metadata Sources**

In [ ]:
!pip install requests

In [ ]:
# =========================================================
# TASK 1 – SUB TASK 2
# UPC → ISBN Resolution (Hybrid: Open Library + Google Books)
# With Progress Tracking
# =========================================================
# This script attempts to resolve ISBN numbers for each book
# using a hybrid lookup strategy:
# 1) Primary source → Open Library (UPC-based search)
# 2) Fallback source → Google Books (Title-based search)
# Results are stored separately for found and not-found ISBNs.

import requests
# requests → used to make HTTP API calls to Open Library and Google Books

import pandas as pd
# pandas → used for reading input CSV, processing rows, and saving outputs

import time
# time → used to introduce delays between API requests (rate limiting)

from pathlib import Path
# Path → OS-independent file and directory handling


# =========================
# CONFIGURATION
# =========================

INPUT_CSV = Path("output/task1_subtask1_clean_books.csv")
# Input CSV containing cleaned book data with UPCs

OUTPUT_DIR = Path("output")
# Directory to store ISBN resolution results

OUTPUT_DIR.mkdir(exist_ok=True)
# Ensures output directory exists

FOUND_CSV = OUTPUT_DIR / "task1_subtask2_isbn_found.csv"
FOUND_JSON = OUTPUT_DIR / "task1_subtask2_isbn_found.json"
# Output files for successfully resolved ISBN records

NOT_FOUND_CSV = OUTPUT_DIR / "task1_subtask2_isbn_not_found.csv"
NOT_FOUND_JSON = OUTPUT_DIR / "task1_subtask2_isbn_not_found.json"
# Output files for records where ISBN could not be resolved

HEADERS = {"User-Agent": "Mozilla/5.0"}
# HTTP headers to simulate a browser request

REQUEST_DELAY = 0.5  # seconds
# Delay between API requests to avoid rate limiting or blocking


# =========================
# HELPER FUNCTIONS
# =========================

def query_open_library(upc):
    """
    Queries Open Library search API using UPC.
    Returns a list of ISBNs and the source name if found.
    """

    url = f"https://openlibrary.org/search.json?q={upc}"
    # Constructs Open Library search URL with UPC as query

    r = requests.get(url, headers=HEADERS, timeout=10)
    # Sends GET request with timeout protection

    if r.status_code != 200:
        return None, None
        # Handles failed HTTP responses gracefully

    data = r.json()
    # Parses JSON response

    if data.get("numFound", 0) == 0:
        return None, None
        # Handles case where no matching records are found

    doc = data["docs"][0]
    # Selects the first matching result

    return doc.get("isbn", []), "openlibrary"
    # Returns ISBN list and source identifier


def query_google_books(query):
    """
    Queries Google Books API using book title as fallback.
    Returns industry identifiers if found.
    """

    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
    # Constructs Google Books API query URL

    r = requests.get(url, headers=HEADERS, timeout=10)
    # Sends GET request with timeout protection

    if r.status_code != 200:
        return None, None
        # Handles failed API calls

    data = r.json()
    # Parses JSON response

    items = data.get("items")
    if not items:
        return None, None
        # Handles case where no results are returned

    identifiers = items[0]["volumeInfo"].get("industryIdentifiers", [])
    # Extracts ISBN identifiers from volume metadata

    return identifiers, "google_books"
    # Returns identifier list and source name


def extract_isbn(isbn_list):
    """
    Extracts ISBN-13 and ISBN-10 from mixed identifier formats.
    Supports both dict-based and string-based ISBN formats.
    """

    isbn_13 = None
    isbn_10 = None

    for i in isbn_list:
        # Iterates over identifiers list

        if isinstance(i, dict):
            # Handles Google Books identifier format

            if i.get("type") == "ISBN_13":
                isbn_13 = i.get("identifier")
            elif i.get("type") == "ISBN_10":
                isbn_10 = i.get("identifier")

        elif isinstance(i, str):
            # Handles Open Library string-based ISBNs

            if len(i) == 13:
                isbn_13 = i
            elif len(i) == 10:
                isbn_10 = i

    return isbn_13, isbn_10
    # Returns extracted ISBNs (if any)


# =========================
# LOAD INPUT DATA
# =========================

df = pd.read_csv(INPUT_CSV)
# Loads cleaned book data into a DataFrame

total_records = len(df)
# Stores total number of records for progress tracking

print(f"📘 Records to process: {total_records}")
# Displays total workload

found_records = []
not_found_records = []
# Lists to store resolved and unresolved ISBN records


# =========================
# PROCESS EACH BOOK
# =========================

for idx, row in df.iterrows():
    # Iterates through each book record

    progress_pct = ((idx + 1) / total_records) * 100
    # Calculates progress percentage

    print(
        f"🔄 Processing {idx + 1}/{total_records} "
        f"({progress_pct:.2f}%) | UPC: {row['UPC']}"
    )
    # Displays progress status with UPC

    upc = row["UPC"]
    title = row["Book Title"]
    # Extracts UPC and title for lookup

    # Primary lookup: Open Library
    isbn_list, source = query_open_library(upc)
    isbn_13, isbn_10 = extract_isbn(isbn_list) if isbn_list else (None, None)
    # Attempts ISBN resolution using UPC via Open Library

    # Fallback: Google Books
    if not isbn_13:
        identifiers, source = query_google_books(title)
        isbn_13, isbn_10 = extract_isbn(identifiers) if identifiers else (None, None)
        # Uses title-based search if UPC lookup fails

    # Store results
    if isbn_13:
        found_records.append({
            **row,
            "ISBN_13": isbn_13,
            "ISBN_10": isbn_10,
            "ISBN_Source": source,
            "ISBN_Status": "success" if source == "openlibrary" else "fallback_used"
        })
        # Stores successfully resolved ISBNs with metadata
    else:
        not_found_records.append({
            **row,
            "ISBN_Reason": "not_found"
        })
        # Stores records where ISBN could not be resolved

    time.sleep(REQUEST_DELAY)
    # Enforces delay to avoid API throttling


# =========================
# SAVE OUTPUT FILES
# =========================

df_found = pd.DataFrame(found_records)
df_not_found = pd.DataFrame(not_found_records)
# Converts results into DataFrames

df_found.to_csv(FOUND_CSV, index=False)
df_found.to_json(FOUND_JSON, orient="records", indent=2)
# Saves successfully resolved ISBN records

df_not_found.to_csv(NOT_FOUND_CSV, index=False)
df_not_found.to_json(NOT_FOUND_JSON, orient="records", indent=2)
# Saves unresolved ISBN records


# =========================
# CONSOLE SAMPLE OUTPUT
# =========================

print("\n✅ ISBN FOUND SAMPLE (first 5 rows):")
print(df_found.head())
# Displays sample of resolved ISBN records

print("\n❌ ISBN NOT FOUND SAMPLE (first 5 rows):")
print(df_not_found.head())
# Displays sample of unresolved ISBN records

print("\n📁 Files generated:")
print(f"➡ {FOUND_CSV}")
print(f"➡ {FOUND_JSON}")
print(f"➡ {NOT_FOUND_CSV}")
print(f"➡ {NOT_FOUND_JSON}")
# Displays generated file paths


# =========================
# CONVERSION SUMMARY
# =========================

print("\n📊 CONVERSION SUMMARY")
print(f"✅ ISBN Converted     : {len(df_found)}")
print(f"❌ ISBN Not Converted : {len(df_not_found)}")
print(f"📘 Total Processed    : {len(df_found) + len(df_not_found)}")
# Prints final conversion statistics

FileNotFoundError: [Errno 2] No such file or directory: 'output/task1_subtask1_clean_books.csv'

#  Observations for **TASK 1 – SUB TASK 2 (UPC → ISBN Resolution)**
---

## 1️⃣ Purpose & Pipeline Role Observation

* This task **enriches cleaned book data** by resolving **ISBN identifiers** using external knowledge sources.
* It acts as a **data augmentation stage**, bridging:

  ```
  Scraped Metadata (UPC, Title)
           ↓
     Global Book Identifiers (ISBN-10 / ISBN-13)
  ```

✅ This makes the dataset **globally identifiable and interoperable**.

---

## 2️⃣ Hybrid Lookup Strategy Observation

* The script uses a **two-level resolution strategy**:

### 🔹 Primary Source

* **Open Library**
* UPC-based search:

  * More **precise**
  * Less ambiguous
  * Faster resolution when available

### 🔹 Fallback Source

* **Google Books**
* Title-based search:

  * Handles missing UPC mappings
  * Improves overall coverage

📌 Observation:

> Using a fallback prevents large data loss when a single source fails.

✅ This is a **robust, production-grade lookup design**.

---

## 3️⃣ Identifier Extraction Observation

* The `extract_isbn()` function correctly handles:

  * Open Library → string-based ISBNs
  * Google Books → dictionary-based identifiers

Supports:

* ISBN-13 (preferred modern standard)
* ISBN-10 (legacy compatibility)

✅ This ensures **format-agnostic ISBN resolution**.

---

## 4️⃣ Progress Tracking & Transparency Observation

* The script prints real-time progress:

  ```text
  Processing 327/1000 (32.70%) | UPC: xxxx
  ```

📌 Benefits:

* Long-running job visibility
* Easy debugging if execution stops
* Suitable for large datasets

✅ Shows **operational awareness**, not just scripting.

---

## 5️⃣ Rate Limiting & API Safety Observation

* A deliberate delay is enforced:

  ```python
  time.sleep(0.5)
  ```

📌 Observation:

* Prevents IP blocking
* Respects public API usage limits
* Increases execution stability

✅ Demonstrates **ethical and responsible API usage**.

---

## 6️⃣ Result Classification Observation

* Outputs are **cleanly separated** into two datasets:

### ✅ ISBN Found

Includes:

* ISBN_13
* ISBN_10 (if available)
* Source (`openlibrary` or `google_books`)
* Resolution status (`success` / `fallback_used`)

### ❌ ISBN Not Found

Includes:

* Original book metadata
* Explicit reason (`not_found`)

📌 Observation:

* No silent failures
* Every record is accounted for

✅ This is critical for **data auditability**.

---

## 7️⃣ Output Design Observation

* Four structured outputs are generated:

  * Found → CSV + JSON
  * Not Found → CSV + JSON

📁 This enables:

* Downstream analytics
* Manual inspection
* Retry or alternative resolution strategies

✅ Excellent **data engineering discipline**.

---

## 8️⃣ Conversion Summary Observation

* Final console summary guarantees:

  ```text
  ISBN Converted + ISBN Not Converted = Total Processed
  ```

📌 Observation:

* No data leakage
* End-to-end accountability

✅ Confirms **pipeline correctness**.

---

## 9️⃣ Expected Real-World Behavior Observation

* Not all books resolve to ISBNs because:

  * Some titles are fictional/demo entries
  * UPC ↔ ISBN mapping may not exist publicly
  * Title ambiguity in Google Books

📌 This is **expected and realistic**, not a flaw.

**Task 2 Web or API query and find the price of the book in competitor website**

In [ ]:
!pip install pandas requests tqdm

In [ ]:
# =========================================================
# TASK 2 – Competitor Price Extraction (BooksRun Internal API)
# =========================================================
# This script fetches competitor book prices from the
# BooksRun internal pricing API using ISBN numbers.
# It compares competitor prices with our price and
# stores structured results for analysis.

import requests
# requests → used to make HTTP calls to the BooksRun API

import pandas as pd
# pandas → used for reading input data, processing rows,
# and saving structured output files

import time
# time → used to introduce delays between API calls (rate limiting)

import json
# json → used implicitly for handling JSON API responses

from pathlib import Path
# Path → ensures OS-independent file path handling

from tqdm import tqdm
# tqdm → provides a progress bar for long-running loops


# =========================
# CONFIGURATION
# =========================

INPUT_CSV = Path("output/task1_subtask2_isbn_found.csv")
# Input file containing books with successfully resolved ISBNs

OUTPUT_DIR = Path("output")
# Directory where Task 2 outputs will be stored

OUTPUT_DIR.mkdir(exist_ok=True)
# Creates output directory if it does not already exist

OUTPUT_CSV = OUTPUT_DIR / "task2_booksrun_competitor_prices.csv"
OUTPUT_JSON = OUTPUT_DIR / "task2_booksrun_competitor_prices.json"
# Output files for competitor pricing data

BOOKSRUN_API_TEMPLATE = (
    "https://booksrun.com/api/v3/price/buy/{isbn}?key=6nszyk9boeq6yad0hqh4"
)
# API endpoint template for BooksRun price lookup using ISBN

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json",
    "Referer": "https://booksrun.com/"
}
# HTTP headers required to access BooksRun API successfully

REQUEST_DELAY = 0.6  # seconds (important to avoid blocking)
# Delay between API requests to avoid being rate-limited or blocked


# =========================
# HELPER FUNCTION
# =========================

def extract_prices(api_response):
    """
    Extracts competitor prices from BooksRun API response.
    Currently focuses on BooksRun's direct 'new' book offers.
    """

    prices = []
    # List to collect extracted prices

    result = api_response.get("result", {})
    offers = result.get("offers", {})
    # Safely navigates nested API response structure

    # =========================
    # BooksRun direct offers
    # =========================
    booksrun = offers.get("booksrun", {})
    # Extracts BooksRun's own offers section

    # New book price
    new_offer = booksrun.get("new")
    # Retrieves new-condition book offer

    if isinstance(new_offer, dict):
        price = new_offer.get("price")
        # Extracts price field

        if isinstance(price, (int, float)):
            prices.append(price)
            # Stores valid numeric price

    return prices
    # Returns list of extracted competitor prices


# =========================
# LOAD INPUT DATA
# =========================

df = pd.read_csv(INPUT_CSV)
# Loads ISBN-resolved book data into DataFrame

total_records = len(df)
# Stores total number of records for summary reporting

print(f"📘 Records to process: {total_records}")
# Displays number of books to be processed

results = []
# List to store final pricing comparison results

found_count = 0
not_found_count = 0
# Counters for summary statistics


# =========================
# PROCESS EACH ISBN
# =========================

for _, row in tqdm(df.iterrows(), total=total_records):
    # Iterates through each book record with progress bar

    isbn13 = str(row["ISBN_13"]).strip()
    isbn10 = str(row.get("ISBN_10", "")).strip()
    # Extracts ISBN-13 and ISBN-10 safely as strings

    api_url = BOOKSRUN_API_TEMPLATE.format(isbn=isbn13)
    # Builds BooksRun API URL for current ISBN

    print(api_url)
    # Prints API URL for debugging / traceability

    competitor_prices = []
    # List to store extracted competitor prices

    try:
        response = requests.get(api_url, headers=HEADERS, timeout=15)
        # Sends API request with timeout protection

        if response.status_code == 200:
            data = response.json()
            # Parses JSON response

            competitor_prices = extract_prices(data)
            # Extracts prices using helper function

    except Exception:
        competitor_prices = []
        # Gracefully handles network or parsing errors

    if competitor_prices:
        found_count += 1
        price_status = "found"
        # Marks competitor price availability

        lowest_price = min(competitor_prices)
        highest_price = max(competitor_prices)
        # Computes price range
    else:
        not_found_count += 1
        price_status = "not_found"
        # Marks competitor price absence

        lowest_price = None
        highest_price = None
        # Uses None when no competitor prices exist

    results.append({
        "Category": row["Category"],
        "Book_Title": row["Book Title"],
        "ISBN_13": isbn13,
        "ISBN_10": isbn10,
        "Our_Price": row["Price"],
        "Competitor_Prices": competitor_prices,
        "Lowest_Competitor_Price": lowest_price,
        "Highest_Competitor_Price": highest_price,
        "Offer_Count": len(competitor_prices),
        "Price_Source": "booksrun_internal_api",
        "Price_Status": price_status
    })
    # Stores structured comparison result for the book

    time.sleep(REQUEST_DELAY)
    # Enforces delay to respect API rate limits


# =========================
# SAVE OUTPUT FILES
# =========================

df_out = pd.DataFrame(results)
# Converts collected results into DataFrame

df_out.to_csv(OUTPUT_CSV, index=False)
# Saves competitor pricing data to CSV

df_out.to_json(OUTPUT_JSON, orient="records", indent=2)
# Saves competitor pricing data to JSON


# =========================
# CONSOLE SUMMARY
# =========================

print("\n📊 TASK 2 SUMMARY")
print(f"✅ Competitor prices found     : {found_count}")
print(f"❌ Competitor prices not found : {not_found_count}")
print(f"📘 Total processed             : {total_records}")
# Displays summary statistics

print("\n📄 SAMPLE OUTPUT (first 5 rows):")
print(df_out.head())
# Displays sample output rows for verification

print("\n📁 Files generated:")
print(f"➡ CSV  : {OUTPUT_CSV}")
print(f"➡ JSON : {OUTPUT_JSON}")
# Displays generated output file paths

📘 Records to process: 907


  0%|          | 0/907 [00:00<?, ?it/s]

https://booksrun.com/api/v3/price/buy/9780062999702?key=6nszyk9boeq6yad0hqh4


  0%|          | 1/907 [00:00<12:59,  1.16it/s]

https://booksrun.com/api/v3/price/buy/9780748129324?key=6nszyk9boeq6yad0hqh4


...

https://booksrun.com/api/v3/price/buy/9780606277747?key=6nszyk9boeq6yad0hqh4


100%|█████████▉| 906/907 [11:50<00:00,  1.30it/s]

https://booksrun.com/api/v3/price/buy/9780761148296?key=6nszyk9boeq6yad0hqh4


100%|██████████| 907/907 [11:51<00:00,  1.28it/s]


📊 TASK 2 SUMMARY
✅ Competitor prices found     : 91
❌ Competitor prices not found : 816
📘 Total processed             : 907

📄 SAMPLE OUTPUT (first 5 rows):
             Category                             Book_Title        ISBN_13  \
0              Poetry                   A Light in the Attic  9780062999702   
1  Historical Fiction                     Tipping the Velvet  9780748129324   
2             Fiction                             Soumission  9789004498136   
3             Mystery                          Sharp Objects  9780307351487   
4             History  Sapiens: A Brief History of Humankind  9780062316103   

      ISBN_10 Our_Price Competitor_Prices  Lowest_Competitor_Price  \
0  0062999702   Â£51.77                []                      NaN   
1  0748129324   Â£53.74                []                      NaN   
2  9004498133   Â£50.10                []                      NaN   
3  0307351483   Â£47.82                []                      NaN   
4  0062316109   Â

#Observations for **TASK 2 – Competitor Price Extraction (BooksRun API)**
---

## 1️⃣ Role of This Task in the Pipeline

* This task performs **competitive price intelligence** using ISBNs resolved in Task 1.
* It connects **internal book pricing** with **external competitor pricing** from
  **BooksRun**.

📌 Observation:

> The pipeline has now moved from *data collection* to *market comparison*.

✅ This is a **business-driven analytics step**, not just scraping.

---

## 2️⃣ Input Coverage Observation

* Total records processed:

```text
Total processed : 907
```

📌 Interpretation:

* Only books with **successfully resolved ISBNs** are passed into Task 2.
* Books without ISBNs are **correctly excluded**, avoiding invalid API calls.

✅ Confirms **clean dependency management between tasks**.

---

## 3️⃣ API Integration Observation

* The script calls an **internal pricing API** endpoint of BooksRun:

```text
https://booksrun.com/api/v3/price/buy/{isbn}
```

📌 Observation:

* Requests include:

  * Custom headers
  * Timeout protection
  * Rate limiting (0.6s delay)

✅ This shows **production-safe API consumption**, not brute-force scraping.

---

## 4️⃣ Price Extraction Logic Observation

* The script:

  * Focuses on **BooksRun’s direct “new” offers**
  * Extracts only **valid numeric prices**
  * Ignores malformed or missing values

📌 Observation:

> This avoids noisy third-party sellers and keeps pricing comparison consistent.

✅ Correct design choice for **controlled competitor benchmarking**.

---

## 5️⃣ Real Output Distribution Observation

From the console summary:

```text
Competitor prices found     : 91
Competitor prices not found : 816
Total processed             : 907
```

📊 Interpretation:

* ~**10%** of books have competitor pricing available
* ~**90%** do not

📌 This is **expected behavior**, because:

* Not all ISBNs are actively traded
* Some books are out of stock or unavailable
* API exposes only certain inventory types

✅ This reflects **real-world market sparsity**, not a failure.

---

## 6️⃣ Output Structure Observation

Each output row contains:

* Book metadata (Category, Title)
* ISBN identifiers
* Our price
* Competitor prices (list)
* Lowest & highest competitor price
* Offer count
* Price source
* Price status (`found` / `not_found`)

📌 Observation:

> Even when prices are missing, rows remain structurally complete.

✅ This guarantees **schema consistency for downstream analysis**.

---

## 7️⃣ Sample Output Insight

From the sample rows:

* `Competitor_Prices` is often an empty list `[]`
* `Lowest_Competitor_Price` and `Highest_Competitor_Price` become `NaN`
* `Offer_Count` is `0`
* `Price_Status` = `not_found`

📌 Observation:

> Missing competitor data is **explicitly represented**, not hidden.

✅ This is essential for **transparent analytics and decision-making**.

---

## 8️⃣ Progress Tracking & Execution Stability

* Uses `tqdm` progress bar
* Prints each API URL
* Handles network/API failures gracefully

📌 Observation:

> The script is suitable for **long-running batch jobs** with full visibility.

✅ Strong operational robustness.

---

## 9️⃣ Output File Generation Observation

* Two final outputs are generated:

```text
output/task2_booksrun_competitor_prices.csv
output/task2_booksrun_competitor_prices.json
```

📌 Observation:

* CSV → analytics, ML, Excel
* JSON → APIs, dashboards, web apps

✅ Demonstrates **multi-consumer data design**.

**Task 3 compare the price in competitor and our website**

In [ ]:
!pip install pandas

In [ ]:
# =========================================================
# TASK 3 – Price Comparison (Our Price vs Competitor Price)
# =========================================================
# This task compares our book price with competitor prices
# to determine whether each book is underpriced, overpriced,
# or priced at parity with the market.

import pandas as pd
# pandas → used for data loading, transformation, and analysis

from pathlib import Path
# Path → provides OS-independent file path handling


# =========================
# CONFIGURATION
# =========================

INPUT_CSV = Path("output/task2_booksrun_competitor_prices.csv")
# Input file generated from Task 2 containing competitor prices

OUTPUT_DIR = Path("output")
# Directory where Task 3 outputs will be stored

OUTPUT_DIR.mkdir(exist_ok=True)
# Ensures the output directory exists

OUTPUT_CSV = OUTPUT_DIR / "task3_price_comparison.csv"
OUTPUT_JSON = OUTPUT_DIR / "task3_price_comparison.json"
# Output files for price comparison results


# =========================
# LOAD DATA
# =========================

df = pd.read_csv(INPUT_CSV)
# Loads competitor pricing data into a DataFrame

# Keep only records where competitor price exists
df = df[df["Price_Status"] == "found"].copy()
# Filters out books without competitor price data

print(f"📘 Records selected for Task 3: {len(df)}")
# Displays number of books eligible for comparison


# =========================
# CLEAN OUR PRICE (FIX ENCODING ISSUE)
# =========================

df["Our_Price_Num"] = (
    df["Our_Price"]
    .astype(str)
    # Converts price to string for safe text operations

    .str.replace("£", "", regex=False)
    # Removes pound symbol from price string

    .str.replace("Â", "", regex=False)
    # Fixes encoding artifact sometimes introduced during scraping

    .astype(float)
    # Converts cleaned price to numeric float
)

# =========================
# PRICE COMPARISON LOGIC
# =========================

df["Price_Difference"] = df["Our_Price_Num"] - df["Lowest_Competitor_Price"]
# Calculates absolute price difference between our price and lowest competitor price

df["Price_Difference_Percent"] = (
    (df["Price_Difference"] / df["Lowest_Competitor_Price"]) * 100
).round(2)
# Calculates percentage difference and rounds to 2 decimal places

def classify_price(diff):
    # Categorizes pricing status based on price difference
    if diff < 0:
        return "underpriced"
    elif diff > 0:
        return "overpriced"
    else:
        return "price_match"

df["Pricing_Status"] = df["Price_Difference"].apply(classify_price)
# Applies classification logic to each book


# =========================
# FINAL OUTPUT COLUMNS
# =========================

final_columns = [
    "Category",
    "Book_Title",
    "ISBN_13",
    "ISBN_10",
    "Our_Price",
    "Lowest_Competitor_Price",
    "Highest_Competitor_Price",
    "Offer_Count",
    "Price_Difference",
    "Price_Difference_Percent",
    "Pricing_Status"
]
# Defines the final schema for Task 3 output

df_final = df[final_columns]
# Selects only relevant columns for output


# =========================
# SAVE OUTPUT FILES
# =========================

df_final.to_csv(OUTPUT_CSV, index=False)
# Saves final comparison results to CSV

df_final.to_json(OUTPUT_JSON, orient="records", indent=2)
# Saves final comparison results to JSON


# =========================
# CONSOLE SUMMARY
# =========================

print("\n📊 TASK 3 SUMMARY")
print("Underpriced books :", (df_final["Pricing_Status"] == "underpriced").sum())
# Counts books cheaper than competitors

print("Overpriced books  :", (df_final["Pricing_Status"] == "overpriced").sum())
# Counts books more expensive than competitors

print("Price matched     :", (df_final["Pricing_Status"] == "price_match").sum())
# Counts books priced equal to competitors

print("\n📄 SAMPLE OUTPUT (first 5 rows):")
print(df_final.head())
# Displays sample comparison results

print("\n📁 Files generated:")
print(f"➡ CSV  : {OUTPUT_CSV}")
print(f"➡ JSON : {OUTPUT_JSON}")
# Displays output file locations

📘 Records selected for Task 3: 91

📊 TASK 3 SUMMARY
Underpriced books : 24
Overpriced books  : 67
Price matched     : 0

📄 SAMPLE OUTPUT (first 5 rows):
          Category                                         Book_Title  \
7          Default  The Coming Woman: A Novel Based on the Life of...   
21  Food and Drink  Foolproof Preserving: A Guide to Small Batch J...   
65    Spirituality  The Activist's Tao Te Ching: Ancient Advice fo...   
69         Fantasy      Saga, Volume 6 (Saga (Collected Editions) #6)   
70  Sequential Art      Saga, Volume 5 (Saga (Collected Editions) #5)   

          ISBN_13     ISBN_10 Our_Price  Lowest_Competitor_Price  \
7   9781941644119  1941644112   Â£17.93                    19.01   
21  9781940352510  1940352517   Â£30.52                    26.40   
65  9781608683925  1608683923   Â£32.24                    13.86   
69  9781506715896  1506715893   Â£25.02                    49.69   
70  9781506715896  1506715893   Â£51.04                    49.69   


#Observations for **TASK 3 – Price Comparison (Our Price vs Competitor Price)**
---

## 1️⃣ Role of Task 3 in the Pipeline

* This task performs **price intelligence analysis** by comparing:

  * **Our listed price**
  * **Lowest available competitor price**
* It transforms raw competitor data into **actionable pricing insights**.

📌 Observation:

> The pipeline has now shifted from *data collection* to *decision-support analytics*.

---

## 2️⃣ Input Data Filtering Observation

* From Task 2:

  * Only books with competitor prices are selected:

```text
📘 Records selected for Task 3: 91
```

📌 Interpretation:

* Out of all processed books, only **91 books (~10%)** had valid competitor prices.
* Books without competitor data are **intentionally excluded** to avoid invalid comparisons.

✅ This ensures **statistically valid price comparison**.

---

## 3️⃣ Price Cleaning & Encoding Fix Observation

* The script explicitly fixes scraping-related encoding issues:

```python
.str.replace("£", "")
.str.replace("Â", "")
```

📌 Observation:

* Handles UTF-8 / ISO-8859-1 encoding artifacts.
* Converts prices safely into numeric floats.

✅ This step is **critical** for accurate mathematical comparison.

---

## 4️⃣ Price Difference Computation Observation

* Two metrics are computed:

  * **Absolute difference**
  * **Percentage difference**

```text
Price_Difference = Our_Price − Lowest_Competitor_Price
```

📌 Observation:

* Absolute difference → direct pricing gap
* Percentage difference → relative competitiveness

✅ Enables **both operational and strategic analysis**.

---

## 5️⃣ Pricing Classification Logic Observation

Books are categorized into:

* `underpriced` → our price < competitor
* `overpriced` → our price > competitor
* `price_match` → equal pricing

📌 Observation:

* Classification is **simple, interpretable, and business-friendly**.
* No arbitrary thresholds → transparent logic.

---

## 6️⃣ Final Pricing Distribution Insight

From the summary:

```text
Underpriced books : 24
Overpriced books  : 67
Price matched     : 0
```

📊 Interpretation:

* **~74% overpriced** → potential loss of competitiveness
* **~26% underpriced** → possible margin sacrifice or aggressive pricing
* **0 price matches** → no parity with competitors

📌 Key Business Insight:

> Pricing is largely **misaligned with the market**, skewed towards being overpriced.

---

## 7️⃣ Sample Output Pattern Observation

From sample rows:

* Same ISBN can appear multiple times with different prices
* `Offer_Count` is usually `1`
* Large percentage differences (e.g. **132% overpriced**) exist

📌 Observation:

* Highlights **significant pricing inefficiencies**
* Identifies **high-risk SKUs** for immediate review

---

## 8️⃣ Output Schema Design Observation

Final output contains:

* Metadata (Category, Title, ISBNs)
* Pricing metrics
* Competitive context
* Final pricing status

📌 Observation:

* Schema is **dashboard-ready**
* Can directly feed:

  * Pricing engines
  * BI tools
  * Recommendation systems

---

## 9️⃣ File Generation & Reusability Observation

* Outputs stored in:

```text
task3_price_comparison.csv
task3_price_comparison.json
```

📌 Observation:

* CSV → analytics / Excel / ML
* JSON → APIs / web apps

✅ Supports **multi-consumer data usage**.

**Task 4 reduce or increase the prices according to competitor**

In [ ]:
!pip install pandas

In [ ]:
# =========================================================
# TASK 4 – Dynamic Price Adjustment (Rule-Based Pricing)
# =========================================================
# This task applies rule-based pricing logic to recommend
# new prices based on competitor comparison results.
# The goal is to stay competitive while following
# predefined business pricing rules.

import pandas as pd
# pandas → used for data manipulation, calculations, and saving results

from pathlib import Path
# Path → provides OS-independent file path handling


# =========================
# CONFIGURATION
# =========================

INPUT_CSV = Path("output/task3_price_comparison.csv")
# Input file generated from Task 3 containing price comparison results

OUTPUT_DIR = Path("output")
# Directory where Task 4 outputs will be stored

OUTPUT_DIR.mkdir(exist_ok=True)
# Ensures output directory exists

OUTPUT_CSV = OUTPUT_DIR / "task4_price_adjustments.csv"
OUTPUT_JSON = OUTPUT_DIR / "task4_price_adjustments.json"
# Output files for final price adjustment recommendations

# Pricing strategy parameters
DECREASE_FACTOR = 0.02   # Reduce price by 2% below competitor price
INCREASE_FACTOR = 0.95   # Increase price towards competitor (95% gap closure)
# Business rules that control how prices are adjusted


# =========================
# LOAD DATA
# =========================

df = pd.read_csv(INPUT_CSV)
# Loads Task 3 price comparison data into a DataFrame

print(f"📘 Records loaded for Task 4: {len(df)}")
# Displays number of records processed in Task 4


# =========================
# CLEAN OUR PRICE
# =========================

df["Our_Price_Num"] = (
    df["Our_Price"]
    .astype(str)
    # Converts price to string for safe text cleaning

    .str.replace("£", "", regex=False)
    # Removes pound currency symbol

    .str.replace("Â", "", regex=False)
    # Removes encoding artifacts from scraped text

    .astype(float)
    # Converts cleaned price back to numeric format
)


# =========================
# PRICE ADJUSTMENT LOGIC
# =========================

def recommend_price(row):
    """
    Determines the recommended price based on pricing status.
    Returns pricing action, new price, adjustment amount, and percent.
    """

    # Pricing decision based on competitor comparison outcome

    our_price = row["Our_Price_Num"]
    competitor_price = row["Lowest_Competitor_Price"]
    # Extracts current and competitor prices
    if row["Pricing_Status"] == "overpriced":
        # If our price is higher than competitor
        new_price = competitor_price - round(competitor_price * DECREASE_FACTOR, 2)
        action = "decrease"

    elif row["Pricing_Status"] == "underpriced":
        # If our price is lower than competitor
        new_price = our_price + round((competitor_price - our_price)* INCREASE_FACTOR, 2)
        action = "increase"

    else:
        # If prices match, keep current price
        new_price = our_price
        action = "no_change"

    adjustment_amount = round(new_price - our_price, 2)
    # Calculates absolute price change

    adjustment_percent = round((adjustment_amount / our_price) * 100, 2)
    # Calculates percentage price change

    return pd.Series([
        action,
        new_price,
        adjustment_amount,
        adjustment_percent
    ])
    # Returns results as a Series for DataFrame assignment


df[
    [
        "Adjustment_Action",
        "Recommended_Price",
        "Price_Adjustment_Amount",
        "Price_Adjustment_Percent"
    ]
] = df.apply(recommend_price, axis=1)
# Applies pricing logic row-wise and creates new columns


# =========================
# FINAL OUTPUT COLUMNS
# =========================

final_columns = [
    "Category",
    "Book_Title",
    "ISBN_13",
    "ISBN_10",
    "Our_Price",
    "Lowest_Competitor_Price",
    "Highest_Competitor_Price",
    "Offer_Count",
    "Pricing_Status",
    "Adjustment_Action",
    "Recommended_Price",
    "Price_Adjustment_Amount",
    "Price_Adjustment_Percent"
]
# Defines the final schema for Task 4 output

df_final = df[final_columns]
# Selects only relevant columns for output


# =========================
# SAVE OUTPUT FILES
# =========================

df_final.to_csv(OUTPUT_CSV, index=False)
# Saves final price adjustment recommendations to CSV

df_final.to_json(OUTPUT_JSON, orient="records", indent=2)
# Saves final price adjustment recommendations to JSON


# =========================
# CONSOLE SUMMARY
# =========================

print("\n📊 TASK 4 SUMMARY")
print("Price decreases :", (df_final["Adjustment_Action"] == "decrease").sum())
# Counts books where price should be reduced

print("Price increases :", (df_final["Adjustment_Action"] == "increase").sum())
# Counts books where price should be increased

print("No change       :", (df_final["Adjustment_Action"] == "no_change").sum())
# Counts books where price remains unchanged

print("\n📄 SAMPLE OUTPUT (first 5 rows):")
print(df_final.head())
# Displays sample adjusted pricing records

print("\n📁 Files generated:")
print(f"➡ CSV  : {OUTPUT_CSV}")
print(f"➡ JSON : {OUTPUT_JSON}")
# Displays output file locations

📘 Records loaded for Task 4: 91

📊 TASK 4 SUMMARY
Price decreases : 67
Price increases : 24
No change       : 0

📄 SAMPLE OUTPUT (first 5 rows):
         Category                                         Book_Title  \
0         Default  The Coming Woman: A Novel Based on the Life of...   
1  Food and Drink  Foolproof Preserving: A Guide to Small Batch J...   
2    Spirituality  The Activist's Tao Te Ching: Ancient Advice fo...   
3         Fantasy      Saga, Volume 6 (Saga (Collected Editions) #6)   
4  Sequential Art      Saga, Volume 5 (Saga (Collected Editions) #5)   

         ISBN_13     ISBN_10 Our_Price  Lowest_Competitor_Price  \
0  9781941644119  1941644112   Â£17.93                    19.01   
1  9781940352510  1940352517   Â£30.52                    26.40   
2  9781608683925  1608683923   Â£32.24                    13.86   
3  9781506715896  1506715893   Â£25.02                    49.69   
4  9781506715896  1506715893   Â£51.04                    49.69   

   Highest_Competit

#Observations for **TASK 4 – Dynamic Price Adjustment (Rule-Based Pricing)**
---

## 1️⃣ Role of Task 4 in the End-to-End Pipeline

* Task 4 is the **decision layer** of the pipeline.
* It converts **price comparison insights (Task 3)** into **actionable pricing recommendations**.

📌 Observation:

> This task operationalizes analytics by translating insights into concrete pricing actions.

---

## 2️⃣ Input Scope Observation

* Records processed:

```text
📘 Records loaded for Task 4: 91
```

📌 Interpretation:

* Only books with valid competitor prices are considered.
* Ensures **all pricing decisions are market-informed**.

✅ This avoids arbitrary or speculative price changes.

---

## 3️⃣ Price Normalization Observation

* The script again fixes currency and encoding artifacts:

```python
.str.replace("£", "")
.str.replace("Â", "")
```

📌 Observation:

* Ensures numeric accuracy for calculations.
* Maintains consistency with Task 3.

✅ This reinforces **data hygiene across pipeline stages**.

---

### 4️⃣ Rule-Based Pricing Strategy Observation (Corrected)

Two explicit business rules are applied:

#### 🔻 For Overpriced Books

* New price = **competitor price − 2%**
* Goal: Undercut the competitor slightly to regain competitiveness without drastic cuts

#### 🔺 For Underpriced Books

* New price = **current price + 95% of the gap to competitor price**
* Goal: Increase margins while still pricing just below the competitor

📌 Observation:

> The pricing logic is **gap-aware**, ensuring controlled corrections rather than abrupt price jumps.

---

## 5️⃣ Pricing Action Distribution Insight

From the summary:

```text
Price decreases : 67
Price increases : 24
No change       : 0
```

📊 Interpretation:

* ~74% of books require **price reduction**
* ~26% of books allow **price increase**
* 0 books already optimally priced

📌 Key Insight:

> Current pricing is largely misaligned with the competitive market.

---

## 6️⃣ Sample Output Behavior Observation

From sample rows:

* Significant **downward corrections** for highly overpriced books
  (e.g., −57.88%)
* Some **large upward adjustments** for heavily underpriced items
  (e.g., +88.69%)

📌 Observation:

> The model aggressively corrects extreme pricing gaps.

---

## 7️⃣ Adjustment Transparency Observation

Each record clearly shows:

* Action (`increase`, `decrease`)
* Recommended price
* Absolute adjustment amount
* Percentage adjustment

📌 Observation:

> Pricing decisions are fully explainable and auditable.

✅ This is critical for **business approval and compliance**.

---

## 8️⃣ Output Schema & Reusability Observation

Final dataset includes:

* Original pricing
* Competitive context
* Pricing decision
* Adjustment metrics

📁 Saved as:

```text
task4_price_adjustments.csv
task4_price_adjustments.json
```

📌 Observation:

* Ready for dashboards, reports, or automated repricing systems.

---

## 9️⃣ Business Readiness Observation

* The model is:

  * Deterministic
  * Interpretable
  * Easy to tune via `DECREASE_FACTOR` and `INCREASE_FACTOR`

📌 Observation:

> This is a safe and explainable alternative to black-box pricing models.